In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

In [2]:
train_data = pd.read_csv('train.csv')

In [3]:
test_data = pd.read_csv('test.csv')

In [4]:
train_data.describe()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
count,957919.000000,942672.000000,942729.000000,942428.000000,942359.000000,942514.000000,942398.000000,942415.000000,9.425460e+05,9.426700e+05,...,942554.000000,942420.000000,942509.000000,942686.000000,942481.000000,942360.000000,9.423300e+05,942512.000000,942707.000000,957919.000000
mean,478959.000000,0.090201,0.345964,4068.744207,0.201214,0.304869,-0.071458,1620.843815,3.771642e+05,1.806054e+15,...,-19.926398,2.074530,23.885245,1.748777,63152.973540,1.208876,4.276905e+16,3959.204669,0.559267,0.498492
std,276527.540591,0.043564,0.146251,6415.829440,0.212510,0.145343,2.123777,1276.281403,3.454325e+05,2.335204e+15,...,18.578439,0.895793,45.581360,10.088848,92435.016241,0.114959,6.732441e+16,3155.991777,0.408426,0.499998
min,0.000000,-0.149910,-0.019044,-9421.700000,-0.082122,-0.006990,-12.791000,-224.800000,-2.984300e+04,-1.153300e+15,...,-105.860000,0.277040,-27.691000,-26.589000,-81977.000000,0.905270,-8.944400e+15,-415.240000,-0.151240,0.000000
25%,239479.500000,0.070227,0.283050,418.430000,0.035086,0.240520,-1.120700,481.545000,9.120900e+04,1.153100e+13,...,-28.812000,1.487700,-0.628880,-4.473975,2443.200000,1.146800,2.321100e+14,1306.200000,0.276560,0.000000
50%,478959.000000,0.090135,0.389100,1279.500000,0.137000,0.327790,-0.380110,1446.100000,2.896700e+05,5.043050e+14,...,-14.636000,1.662100,1.727700,0.885710,19479.000000,1.177200,1.327500e+16,3228.000000,0.473440,0.000000
75%,718438.500000,0.116500,0.458450,4444.400000,0.297100,0.412830,0.921940,2495.900000,5.605600e+05,3.103100e+15,...,-5.325300,2.522325,18.991000,6.840775,88488.000000,1.242000,5.278700e+16,6137.900000,0.746210,1.000000
max,957918.000000,0.415170,0.518990,39544.000000,1.319900,0.554750,11.202000,5426.600000,1.913700e+06,1.042400e+16,...,1.613400,4.565900,217.840000,47.757000,526050.000000,1.886700,3.249900e+17,13151.000000,2.743600,1.000000


In [5]:
train_data.shape

(957919, 120)

In [6]:
test_data.shape

(493474, 119)

In [7]:
train_data.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,...,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,...,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,...,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1
3,3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,...,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1
4,4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,...,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1


In [8]:
train_data.isnull().sum()

id           0
f1       15247
f2       15190
f3       15491
f4       15560
         ...  
f115     15559
f116     15589
f117     15407
f118     15212
claim        0
Length: 120, dtype: int64

In [9]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

In [10]:
impured_train = pd.DataFrame(imp_mean.fit_transform(train_data), columns=train_data.columns)
impured_test = pd.DataFrame(imp_mean.fit_transform(test_data), columns=test_data.columns)

In [11]:
impured_train.isnull().sum()

id       0
f1       0
f2       0
f3       0
f4       0
        ..
f115     0
f116     0
f117     0
f118     0
claim    0
Length: 120, dtype: int64

In [12]:
impured_test.isnull().sum()

id      0
f1      0
f2      0
f3      0
f4      0
       ..
f114    0
f115    0
f116    0
f117    0
f118    0
Length: 119, dtype: int64

In [13]:
impured_train.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0.0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,...,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.100000,1.01770,1.0
1,1.0,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,...,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.000000,0.28359,0.0
2,2.0,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,...,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.200000,0.40690,1.0
3,3.0,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,...,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.100000,0.51486,1.0
4,4.0,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,...,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,3959.204669,0.23049,1.0


In [14]:
impured_train_X = impured_train.iloc[:, 1:-1]
impured_test = impured_test.iloc[:, 1:]

impured_train_y = pd.DataFrame(impured_train.iloc[:, 119])

In [15]:
impured_train_X.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,86.489,...,0.11093,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.100000,1.01770
1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,...,0.97673,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.000000,0.28359
2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,15827.000,...,0.20102,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.200000,0.40690
3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,-36.837,...,-0.01182,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.100000,0.51486
4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,144.120,...,0.92739,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,3959.204669,0.23049


In [16]:
train_X, valid_X, train_y, valid_y = train_test_split(impured_train_X, impured_train_y, test_size=0.33)

In [17]:
train_X.shape

(641805, 118)

In [18]:
train_X.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
744430,0.079040,0.282040,1785.20,0.076452,0.317680,3.69650,2018.10,64782.0,2.469200e+14,41.203,...,0.440060,-48.2580,1.9157,110.27000,0.47954,105960.0000,1.1545,-1.203600e+14,2751.20,0.619280
147595,0.071241,0.452710,498.26,0.188330,0.393730,0.48185,-121.21,126070.0,9.185900e+10,58324.000,...,0.054974,-8.5286,1.1393,0.60634,1.65870,312.6500,1.2517,1.883600e+14,999.41,0.078232
264045,0.091745,0.438190,645.89,0.180550,-0.000512,-1.21940,2848.80,584020.0,6.939700e+15,-52.479,...,0.620960,-12.4380,1.2806,18.78900,-6.61470,6963.0000,1.3027,-1.235300e+15,257.96,0.281590
167029,0.079134,0.423360,465.27,0.004043,0.350050,-0.30515,1943.80,319020.0,3.011100e+15,3038.000,...,0.596590,-7.8267,1.4656,1.12500,0.58952,206920.0000,1.3154,1.441900e+17,4676.80,0.470430
585359,0.166010,0.068575,18468.00,0.081399,0.286980,-0.47409,2285.10,19256.0,2.063000e+13,103.130,...,0.264890,-1.6591,1.1793,22.69400,-0.72505,3.9266,1.1895,1.175000e+16,3550.40,0.738530


In [19]:
train_y.head()

,claim
744430,1.0
147595,0.0
264045,0.0
167029,0.0
585359,0.0


In [20]:
train_y.shape

(641805, 1)

In [21]:
class MyRandomForestClassifier:
    def __init__(self, n_estimators=100, criterion='gini',
                 max_depth=None, min_samples_split=2,
                 min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                 max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0,
                 min_impurity_split=None, bootstrap=True, random_state=None, max_samples=None):
        self.n_estimators = n_estimators
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_weight_fraction_leaf = min_weight_fraction_leaf
        self.max_features = max_features
        self.max_leaf_nodes = max_leaf_nodes
        self.min_impurity_decrease = min_impurity_decrease
        # self.min_impurity_split = min_impurity_split
        self.bootstrap = bootstrap
        self.random_state = random_state
        self.max_samples = max_samples

        self.estimators = []
        for i in range(self.n_estimators):
            self.estimators.append(
                DecisionTreeClassifier(criterion=self.criterion, max_depth=self.max_depth,
                                       min_samples_split=self.min_samples_split, min_samples_leaf=self.min_samples_leaf,
                                       min_weight_fraction_leaf=self.min_weight_fraction_leaf,
                                       max_features=self.max_features,
                                       max_leaf_nodes=self.max_leaf_nodes,
                                       min_impurity_decrease=self.min_impurity_decrease,
                                       random_state=self.random_state))

    def fit(self, x, y):
        samples_count = x.shape[0]
        if self.bootstrap is True and self.max_samples is not None:
            samples_count = self.max_samples

        bootstrapped_x = pd.DataFrame()
        bootstrapped_y = pd.DataFrame()

        np.random.seed(self.random_state)
        bootstrapped_rows = np.random.choice(x.shape[0], samples_count, replace=self.bootstrap)
        bootstrapped_x = x.iloc[bootstrapped_rows]
        bootstrapped_y = y.iloc[bootstrapped_rows]

        for clf in self.estimators:
            clf.fit(bootstrapped_x, bootstrapped_y)

    def predict(self, x):
        ones = [0 for i in range(x.shape[0])]

        for i, clf in enumerate(self.estimators):
            predictions = list(clf.predict(x))
            for val in predictions:
                if val == 1:
                    ones[i] += 1

        return pd.Series(ones)

    def predict_proba(self, x):
        ones = [0 for i in range(x.shape[0])]

        for i, clf in enumerate(self.estimators):
            predictions = list(clf.predict(x))
            for val in predictions:
                if val == 1:
                    ones[i] += 1

        ones_probabilities = [p / x.shape[0] for p in ones]
        zeroes_probabilities = [1 - p for p in ones_probabilities]
        probabilities = np.array([zeroes_probabilities, ones_probabilities])
        return probabilities.transpose()

In [22]:
my_clf = MyRandomForestClassifier(n_estimators=10, bootstrap=True, max_samples=1000)

In [23]:
my_clf.fit(train_X, train_y)

In [24]:
y_pred = my_clf.predict(valid_X)

In [25]:
accuracy_score(valid_y, y_pred)

0.5018158006288871

In [26]:
y_pred = my_clf.predict_proba(valid_X)

In [27]:
y_pred

array([[0.53602814, 0.46397186],
       [0.53718279, 0.46281721],
       [0.49672903, 0.50327097],
       ...,
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ]])

In [28]:
roc_auc_score(valid_y, y_pred[:, 1])

0.49999378904832875

In [29]:
rfc_sklearn = RandomForestClassifier(n_estimators=100, criterion='gini', 
                                     max_depth=None, min_samples_split=2, 
                                     min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                     max_features='auto', max_leaf_nodes=None, 
                                     min_impurity_decrease=0.0, min_impurity_split=None,
                                     bootstrap=True, random_state=None, max_samples=None, verbose=2)

In [30]:
rfc_sklearn.fit(train_X, np.ravel(train_y))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.2s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 26.0min finished


RandomForestClassifier(verbose=2)

In [31]:
y_pred = rfc_sklearn.predict(valid_X)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   20.6s finished


In [32]:
accuracy_score(valid_y, y_pred)

0.5453950157221762

In [33]:
y_pred = rfc_sklearn.predict_proba(valid_X)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   17.5s finished


In [34]:
roc_auc_score(valid_y, y_pred[:, 1])

0.5635550536093596

In [35]:
lightgbm_rfc = LGBMClassifier(n_estimators=100, criterion='gini', 
                              max_depth=None, min_samples_split=2, 
                              min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                              max_features='auto', max_leaf_nodes=None, 
                              min_impurity_decrease=0.0, min_impurity_split=None,
                              bootstrap=True, random_state=None, max_samples=None, verbose=0)

In [36]:
lightgbm_rfc.fit(train_X, np.ravel(train_y))

[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: min_impurity_decrease
[LightGBM] [Warning] Unknown parameter: min_weight_fraction_leaf
[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Warning] Unknown parameter: bootstrap
[LightGBM] [Warning] min_data_in_leaf is set with min_samples_leaf=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] Unknown parameter: criterion
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: min_impurity_decrease
[LightGBM] [Warning] Unknown parameter: min_weight_fraction_leaf
[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Warning] Unknown parameter: bootstrap
[LightGBM] [Warning] min_data_in_leaf is set with min_samples_leaf=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] Unknown parameter: criterion
[LightGBM] [Warning] Auto-choosing col-w

LGBMClassifier(bootstrap=True, criterion='gini', max_depth=None,
               max_features='auto', max_leaf_nodes=None, max_samples=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, verbose=0)

In [37]:
y_pred = lightgbm_rfc.predict(valid_X)

In [38]:
accuracy_score(valid_y, y_pred)

0.7034329387499446

In [39]:
y_pred = lightgbm_rfc.predict_proba(valid_X)

In [40]:
roc_auc_score(valid_y, y_pred[:, 1])

0.7655867113897186

In [41]:
feature_importances = lightgbm_rfc.feature_importances_

In [42]:
feature_importances

array([16, 33, 62, 19, 32, 26, 30, 48, 23, 41, 13, 15, 21, 20, 16, 44, 15,
       12, 21,  6, 54,  9, 11, 36, 15, 15, 23, 32, 15, 43, 27, 33, 18, 53,
       58, 45,  4, 17, 44, 80,  2,  8,  9,  9, 65, 34, 68, 34, 16, 37,  5,
       27, 35, 19, 10, 12, 48, 14, 15, 12, 40, 32,  3, 17, 47, 13,  6,  5,
        5, 70, 44,  0, 13, 21, 16,  4, 55, 24, 32,  3, 30,  1, 21, 13, 10,
       21,  7,  5, 13, 20, 44, 61,  9, 29, 49, 52, 37, 16, 38, 16, 17, 31,
        5, 45, 16, 48, 31, 38, 26, 19, 41, 12, 23, 37, 18, 11,  3, 13],
      dtype=int32)

In [43]:
sort_feature_importances = [(imp, idx) for idx, imp in enumerate(feature_importances)]

In [44]:
sort_feature_importances

[(16, 0),
 (33, 1),
 (62, 2),
 (19, 3),
 (32, 4),
 (26, 5),
 (30, 6),
 (48, 7),
 (23, 8),
 (41, 9),
 (13, 10),
 (15, 11),
 (21, 12),
 (20, 13),
 (16, 14),
 (44, 15),
 (15, 16),
 (12, 17),
 (21, 18),
 (6, 19),
 (54, 20),
 (9, 21),
 (11, 22),
 (36, 23),
 (15, 24),
 (15, 25),
 (23, 26),
 (32, 27),
 (15, 28),
 (43, 29),
 (27, 30),
 (33, 31),
 (18, 32),
 (53, 33),
 (58, 34),
 (45, 35),
 (4, 36),
 (17, 37),
 (44, 38),
 (80, 39),
 (2, 40),
 (8, 41),
 (9, 42),
 (9, 43),
 (65, 44),
 (34, 45),
 (68, 46),
 (34, 47),
 (16, 48),
 (37, 49),
 (5, 50),
 (27, 51),
 (35, 52),
 (19, 53),
 (10, 54),
 (12, 55),
 (48, 56),
 (14, 57),
 (15, 58),
 (12, 59),
 (40, 60),
 (32, 61),
 (3, 62),
 (17, 63),
 (47, 64),
 (13, 65),
 (6, 66),
 (5, 67),
 (5, 68),
 (70, 69),
 (44, 70),
 (0, 71),
 (13, 72),
 (21, 73),
 (16, 74),
 (4, 75),
 (55, 76),
 (24, 77),
 (32, 78),
 (3, 79),
 (30, 80),
 (1, 81),
 (21, 82),
 (13, 83),
 (10, 84),
 (21, 85),
 (7, 86),
 (5, 87),
 (13, 88),
 (20, 89),
 (44, 90),
 (61, 91),
 (9, 92),
 (29, 

In [45]:
sort_feature_importances.sort(reverse=True)

In [46]:
sort_feature_importances

[(80, 39),
 (70, 69),
 (68, 46),
 (65, 44),
 (62, 2),
 (61, 91),
 (58, 34),
 (55, 76),
 (54, 20),
 (53, 33),
 (52, 95),
 (49, 94),
 (48, 105),
 (48, 56),
 (48, 7),
 (47, 64),
 (45, 103),
 (45, 35),
 (44, 90),
 (44, 70),
 (44, 38),
 (44, 15),
 (43, 29),
 (41, 110),
 (41, 9),
 (40, 60),
 (38, 107),
 (38, 98),
 (37, 113),
 (37, 96),
 (37, 49),
 (36, 23),
 (35, 52),
 (34, 47),
 (34, 45),
 (33, 31),
 (33, 1),
 (32, 78),
 (32, 61),
 (32, 27),
 (32, 4),
 (31, 106),
 (31, 101),
 (30, 80),
 (30, 6),
 (29, 93),
 (27, 51),
 (27, 30),
 (26, 108),
 (26, 5),
 (24, 77),
 (23, 112),
 (23, 26),
 (23, 8),
 (21, 85),
 (21, 82),
 (21, 73),
 (21, 18),
 (21, 12),
 (20, 89),
 (20, 13),
 (19, 109),
 (19, 53),
 (19, 3),
 (18, 114),
 (18, 32),
 (17, 100),
 (17, 63),
 (17, 37),
 (16, 104),
 (16, 99),
 (16, 97),
 (16, 74),
 (16, 48),
 (16, 14),
 (16, 0),
 (15, 58),
 (15, 28),
 (15, 25),
 (15, 24),
 (15, 16),
 (15, 11),
 (14, 57),
 (13, 117),
 (13, 88),
 (13, 83),
 (13, 72),
 (13, 65),
 (13, 10),
 (12, 111),
 (12,

In [47]:
top_features = [idx for imp, idx in sort_feature_importances]

In [48]:
top_features

[39,
 69,
 46,
 44,
 2,
 91,
 34,
 76,
 20,
 33,
 95,
 94,
 105,
 56,
 7,
 64,
 103,
 35,
 90,
 70,
 38,
 15,
 29,
 110,
 9,
 60,
 107,
 98,
 113,
 96,
 49,
 23,
 52,
 47,
 45,
 31,
 1,
 78,
 61,
 27,
 4,
 106,
 101,
 80,
 6,
 93,
 51,
 30,
 108,
 5,
 77,
 112,
 26,
 8,
 85,
 82,
 73,
 18,
 12,
 89,
 13,
 109,
 53,
 3,
 114,
 32,
 100,
 63,
 37,
 104,
 99,
 97,
 74,
 48,
 14,
 0,
 58,
 28,
 25,
 24,
 16,
 11,
 57,
 117,
 88,
 83,
 72,
 65,
 10,
 111,
 59,
 55,
 17,
 115,
 22,
 84,
 54,
 92,
 43,
 42,
 21,
 41,
 86,
 66,
 19,
 102,
 87,
 68,
 67,
 50,
 75,
 36,
 116,
 79,
 62,
 40,
 81,
 71]

In [49]:
top20_features = top_features[:20]

In [50]:
top20_features

[39,
 69,
 46,
 44,
 2,
 91,
 34,
 76,
 20,
 33,
 95,
 94,
 105,
 56,
 7,
 64,
 103,
 35,
 90,
 70]

In [51]:
impured_train_y

,claim
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
957914,0.0
957915,1.0
957916,0.0
957917,1.0


In [52]:
impured_train_X = impured_train_X.iloc[:, top20_features]
impured_train_y = impured_train_y
impured_test = impured_test.iloc[:, top20_features]

train_X, valid_X, train_y, valid_y = train_test_split(impured_train_X, impured_train_y, test_size=0.33)

In [53]:
rfc_sklearn = RandomForestClassifier(n_estimators=100, criterion='entropy', 
                                     max_depth=None, min_samples_split=2, 
                                     min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                     max_features='auto', max_leaf_nodes=None, 
                                     min_impurity_decrease=0.0, min_impurity_split=None,
                                     bootstrap=True, random_state=None, max_samples=None, verbose=2)

In [54]:
rfc_sklearn.fit(train_X, np.ravel(train_y))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.9s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 11.2min finished


RandomForestClassifier(criterion='entropy', verbose=2)

In [55]:
y_pred = rfc_sklearn.predict_proba(valid_X)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   18.2s finished


In [56]:
roc_auc_score(valid_y, y_pred[:, 1])

0.5731791780303577

LGBM tuning

In [57]:
params = {
    'n_estimators': range(1, 300),
    'max_depth': range(1, 10),
    'min_samples_split': range(2, 50),
    'min_samples_leaf': range(1, 100),
    'max_leaf_nodes': range(10, 200),
    'max_samples': range(10, 10000)
}

In [58]:
clf = RandomizedSearchCV(lightgbm_rfc, params, cv=5, verbose=2)

In [59]:
clf.fit(train_X, np.ravel(train_y))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Warning] Unknown parameter: min_impurity_decrease
[LightGBM] [Warning] Unknown parameter: min_weight_fraction_leaf
[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: max_samples
[LightGBM] [Warning] Unknown parameter: bootstrap
[LightGBM] [Warning] min_data_in_leaf is set with min_samples_leaf=45, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=45
[LightGBM] [Warning] Unknown parameter: criterion
[LightGBM] [Warning] num_leaves is set=31, max_leaf_nodes=106 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] Unknown parameter: min_impurity_decrease
[LightGBM] [Warning] Unknown parameter: min_weight_fraction_leaf
[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: max_samples
[LightG

RandomizedSearchCV(cv=5,
                   estimator=LGBMClassifier(bootstrap=True, criterion='gini',
                                            max_depth=None, max_features='auto',
                                            max_leaf_nodes=None,
                                            max_samples=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            verbose=0),
                   param_distributions={'max_depth': range(1, 10),
                                        'max_leaf_nodes': range(10, 200),
                                        'max_samples': range(10, 10000),
                                        'min_samples_leaf': range(1, 100),
 

In [60]:
clf.best_params_

{'n_estimators': 245,
 'min_samples_split': 41,
 'min_samples_leaf': 28,
 'max_samples': 1454,
 'max_leaf_nodes': 122,
 'max_depth': 6}

In [61]:
my_clf = MyRandomForestClassifier(**clf.best_params_, bootstrap=True)
my_clf.fit(train_X, train_y)

In [62]:
rfc_sklearn = RandomForestClassifier(**clf.best_params_, verbose=2, bootstrap=True)
rfc_sklearn.fit(train_X, np.ravel(train_y))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 245
building tree 2 of 245
building tree 3 of 245
building tree 4 of 245
building tree 5 of 245
building tree 6 of 245
building tree 7 of 245
building tree 8 of 245
building tree 9 of 245
building tree 10 of 245
building tree 11 of 245
building tree 12 of 245
building tree 13 of 245
building tree 14 of 245
building tree 15 of 245
building tree 16 of 245
building tree 17 of 245
building tree 18 of 245
building tree 19 of 245
building tree 20 of 245
building tree 21 of 245
building tree 22 of 245
building tree 23 of 245
building tree 24 of 245
building tree 25 of 245
building tree 26 of 245
building tree 27 of 245
building tree 28 of 245
building tree 29 of 245
building tree 30 of 245
building tree 31 of 245
building tree 32 of 245
building tree 33 of 245
building tree 34 of 245
building tree 35 of 245
building tree 36 of 245
building tree 37 of 245
building tree 38 of 245
building tree 39 of 245
building tree 40 of 245
building tree 41 of 245
building tree 42 of 245
b

[Parallel(n_jobs=1)]: Done 245 out of 245 | elapsed:   15.7s finished


RandomForestClassifier(max_depth=6, max_leaf_nodes=122, max_samples=1454,
                       min_samples_leaf=28, min_samples_split=41,
                       n_estimators=245, verbose=2)

In [63]:
lightgbm_rfc = LGBMClassifier(**clf.best_params_, verbose=2, bootstrap=True)
lightgbm_rfc.fit(train_X, np.ravel(train_y))

[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Warning] Unknown parameter: bootstrap
[LightGBM] [Warning] Unknown parameter: max_samples
[LightGBM] [Warning] min_data_in_leaf is set with min_samples_leaf=28, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=28
[LightGBM] [Warning] num_leaves is set=31, max_leaf_nodes=122 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Warning] Unknown parameter: bootstrap
[LightGBM] [Warning] Unknown parameter: max_samples
[LightGBM] [Warning] min_data_in_leaf is set with min_samples_leaf=28, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=28
[LightGBM] [Warning] num_leaves is set=31, max_leaf_nodes=122 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] Accuracy may be bad since

LGBMClassifier(bootstrap=True, max_depth=6, max_leaf_nodes=122,
               max_samples=1454, min_samples_leaf=28, min_samples_split=41,
               n_estimators=245, verbose=2)

In [64]:
y_pred = my_clf.predict(valid_X)
accuracy_score(valid_y, y_pred)

0.5012938370334753

In [65]:
y_pred = my_clf.predict_proba(valid_X)
roc_auc_score(valid_y, y_pred[:, 1])

0.49999943536741676

In [66]:
y_pred = rfc_sklearn.predict(valid_X)
accuracy_score(valid_y, y_pred)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 245 out of 245 | elapsed:    4.4s finished


0.539182067228911

In [67]:
y_pred = rfc_sklearn.predict_proba(valid_X)
roc_auc_score(valid_y, y_pred[:, 1])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 245 out of 245 | elapsed:    4.4s finished


0.5547687395404384

In [68]:
y_pred = lightgbm_rfc.predict(valid_X)
accuracy_score(valid_y, y_pred)

0.6188305484730192

In [69]:
y_pred = lightgbm_rfc.predict_proba(valid_X)
roc_auc_score(valid_y, y_pred[:, 1])

0.6491807544782857

Sklearn tuning

In [70]:
clf = RandomizedSearchCV(rfc_sklearn, params, cv=5, verbose=2)

In [71]:
clf.fit(train_X, np.ravel(train_y))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 142
building tree 2 of 142
building tree 3 of 142
building tree 4 of 142
building tree 5 of 142
building tree 6 of 142
building tree 7 of 142
building tree 8 of 142
building tree 9 of 142
building tree 10 of 142
building tree 11 of 142
building tree 12 of 142
building tree 13 of 142
building tree 14 of 142
building tree 15 of 142
building tree 16 of 142
building tree 17 of 142
building tree 18 of 142
building tree 19 of 142
building tree 20 of 142
building tree 21 of 142
building tree 22 of 142
building tree 23 of 142
building tree 24 of 142
building tree 25 of 142
building tree 26 of 142
building tree 27 of 142
building tree 28 of 142
building tree 29 of 142
building tree 30 of 142
building tree 31 of 142
building tree 32 of 142
building tree 33 of 142
building tree 34 of 142
building tree 35 of 142
building tree 36 of 142
building tree 37 of 142
building tree 38 of 142
building tree 39 of 142
building tree 40 of 142
building tree 41 of 142
building tree 42 of 142
b

[Parallel(n_jobs=1)]: Done 142 out of 142 | elapsed:    9.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 142 out of 142 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END max_depth=9, max_leaf_nodes=20, max_samples=7199, min_samples_leaf=71, min_samples_split=48, n_estimators=142; total time=  10.5s
building tree 1 of 142


...

[CV] END max_depth=7, max_leaf_nodes=96, max_samples=5271, min_samples_leaf=47, min_samples_split=26, n_estimators=204; total time=  14.8s
building tree 1 of 132


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 2 of 132
building tree 3 of 132
building tree 4 of 132
building tree 5 of 132
building tree 6 of 132
building tree 7 of 132
building tree 8 of 132
building tree 9 of 132
building tree 10 of 132
building tree 11 of 132
building tree 12 of 132
building tree 13 of 132
building tree 14 of 132
building tree 15 of 132
building tree 16 of 132
building tree 17 of 132
building tree 18 of 132
building tree 19 of 132
building tree 20 of 132
building tree 21 of 132
building tree 22 of 132
building tree 23 of 132
building tree 24 of 132
building tree 25 of 132
building tree 26 of 132
building tree 27 of 132
building tree 28 of 132
building tree 29 of 132
building tree 30 of 132
building tree 31 of 132
building tree 32 of 132
building tree 33 of 132
building tree 34 of 132
building tree 35 of 132
building tree 36 of 132
building tree 37 of 132
building tree 38 of 132
building tree 39 of 132
building tree 40 of 132
building tree 41 of 132
building tree 42 of 132
building tree 43 of 132


[Parallel(n_jobs=1)]: Done 132 out of 132 | elapsed:   11.2s finished


RandomizedSearchCV(cv=5,
                   estimator=RandomForestClassifier(max_depth=6,
                                                    max_leaf_nodes=122,
                                                    max_samples=1454,
                                                    min_samples_leaf=28,
                                                    min_samples_split=41,
                                                    n_estimators=245,
                                                    verbose=2),
                   param_distributions={'max_depth': range(1, 10),
                                        'max_leaf_nodes': range(10, 200),
                                        'max_samples': range(10, 10000),
                                        'min_samples_leaf': range(1, 100),
                                        'min_samples_split': range(2, 50),
                                        'n_estimators': range(1, 300)},
                   verbose=2)

In [72]:
clf.best_params_

{'n_estimators': 132,
 'min_samples_split': 20,
 'min_samples_leaf': 39,
 'max_samples': 6944,
 'max_leaf_nodes': 123,
 'max_depth': 9}

In [73]:
rfc_sklearn = RandomForestClassifier(**clf.best_params_, verbose=2, bootstrap=True)
rfc_sklearn.fit(train_X, np.ravel(train_y))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 132
building tree 2 of 132
building tree 3 of 132
building tree 4 of 132
building tree 5 of 132
building tree 6 of 132
building tree 7 of 132
building tree 8 of 132
building tree 9 of 132
building tree 10 of 132
building tree 11 of 132
building tree 12 of 132
building tree 13 of 132
building tree 14 of 132
building tree 15 of 132
building tree 16 of 132
building tree 17 of 132
building tree 18 of 132
building tree 19 of 132
building tree 20 of 132
building tree 21 of 132
building tree 22 of 132
building tree 23 of 132
building tree 24 of 132
building tree 25 of 132
building tree 26 of 132
building tree 27 of 132
building tree 28 of 132
building tree 29 of 132
building tree 30 of 132
building tree 31 of 132
building tree 32 of 132
building tree 33 of 132
building tree 34 of 132
building tree 35 of 132
building tree 36 of 132
building tree 37 of 132
building tree 38 of 132
building tree 39 of 132
building tree 40 of 132
building tree 41 of 132
building tree 42 of 132
b

[Parallel(n_jobs=1)]: Done 132 out of 132 | elapsed:   11.1s finished


RandomForestClassifier(max_depth=9, max_leaf_nodes=123, max_samples=6944,
                       min_samples_leaf=39, min_samples_split=20,
                       n_estimators=132, verbose=2)

In [74]:
y_pred = rfc_sklearn.predict_proba(valid_X)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 132 out of 132 | elapsed:    3.0s finished


In [75]:
roc_auc_score(valid_y, y_pred[:, 1])

0.5631037675170661

In [76]:
%timeit my_clf.predict_proba(valid_X)

27.1 s ± 1.21 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [77]:
%timeit rfc_sklearn.predict_proba(valid_X)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 132 out of 132 | elapsed:    2.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 132 out of 132 | elapsed:    2.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 132 out of 132 | elapsed:    2.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 132 out of 132 | elapsed:    2.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent work

2.9 s ± 6.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[Parallel(n_jobs=1)]: Done 132 out of 132 | elapsed:    2.9s finished


In [78]:
%timeit lightgbm_rfc.predict_proba(valid_X)

1.72 s ± 151 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [79]:
class MyEnsemble:
    def __init__(self, n_estimators=100, criterion='gini',
                 max_depth=None, min_samples_split=2,
                 min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                 max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0,
                 min_impurity_split=None, bootstrap=True, random_state=None, max_samples=None):
        self.n_estimators = n_estimators
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_weight_fraction_leaf = min_weight_fraction_leaf
        self.max_features = max_features
        self.max_leaf_nodes = max_leaf_nodes
        self.min_impurity_decrease = min_impurity_decrease
        # self.min_impurity_split = min_impurity_split
        self.bootstrap = bootstrap
        self.random_state = random_state
        self.max_samples = max_samples

        self.my_random_forest = MyRandomForestClassifier(n_estimators=self.n_estimators, criterion=self.criterion, 
                                                         max_depth=self.max_depth, min_samples_split=self.min_samples_split, 
                                                         min_samples_leaf=self.min_samples_leaf, min_weight_fraction_leaf=self.min_weight_fraction_leaf, 
                                                         max_features=self.max_features, max_leaf_nodes=self.max_leaf_nodes, 
                                                         min_impurity_decrease=self.min_impurity_decrease, bootstrap=self.bootstrap, 
                                                         random_state=self.random_state, max_samples=self.max_samples)
        
        self.sklearn_rfc = RandomForestClassifier(n_estimators=self.n_estimators, criterion=self.criterion, 
                                                  max_depth=self.max_depth, min_samples_split=self.min_samples_split, 
                                                  min_samples_leaf=self.min_samples_leaf, min_weight_fraction_leaf=self.min_weight_fraction_leaf, 
                                                  max_features=self.max_features, max_leaf_nodes=self.max_leaf_nodes, 
                                                  min_impurity_decrease=self.min_impurity_decrease, bootstrap=self.bootstrap, 
                                                  random_state=self.random_state, max_samples=self.max_samples)
        
        self.lightgbm_rfc = LGBMClassifier(n_estimators=self.n_estimators, criterion=self.criterion, 
                                           max_depth=self.max_depth, min_samples_split=self.min_samples_split, 
                                           min_samples_leaf=self.min_samples_leaf, min_weight_fraction_leaf=self.min_weight_fraction_leaf, 
                                           max_features=self.max_features, max_leaf_nodes=self.max_leaf_nodes, 
                                           min_impurity_decrease=self.min_impurity_decrease, bootstrap=self.bootstrap, 
                                           random_state=self.random_state, max_samples=self.max_samples)
        
    def fit(self, x, y):
        samples_count = x.shape[0]
        if self.bootstrap is True and self.max_samples is not None:
            samples_count = self.max_samples

        bootstrapped_x = pd.DataFrame()
        bootstrapped_y = pd.DataFrame()

        bootstrapped_rows = np.random.choice(x.shape[0], samples_count, replace=self.bootstrap)
        bootstrapped_x = x.iloc[bootstrapped_rows]
        bootstrapped_y = y.iloc[bootstrapped_rows]
        
        self.my_random_forest.fit(bootstrapped_x, bootstrapped_y)
        self.sklearn_rfc.fit(bootstrapped_x, np.ravel(bootstrapped_y))
        self.lightgbm_rfc.fit(bootstrapped_x, np.ravel(bootstrapped_y))

    def predict(self, x):
        ones = [0 for i in range(x.shape[0])]

        predictions = list(self.my_random_forest.predict(x))
        for i, val in enumerate(predictions):
            ones[i] += val
            
        predictions = list(self.sklearn_rfc.predict(x))
        for i, val in enumerate(predictions):
            ones[i] += val

        predictions = list(self.lightgbm_rfc.predict(x))
        for i, val in enumerate(predictions):
            ones[i] += val
    
        predictions = [1 if x > 1 else 0 for x in ones]
        return pd.Series(predictions)
    
    def predict_proba(self, x):
        ones_probabilities = [0 for i in range(x.shape[0])]

        predictions = self.my_random_forest.predict_proba(x)
        for i, val in enumerate(predictions[:, 1]):
            ones_probabilities[i] += val
            
        predictions = self.sklearn_rfc.predict_proba(x)
        for i, val in enumerate(predictions[:, 1]):
            ones_probabilities[i] += val

        predictions = self.lightgbm_rfc.predict_proba(x)
        for i, val in enumerate(predictions[:, 1]):
            ones_probabilities[i] += val
        
        ones_probabilities = [x / 3 for x in ones_probabilities]
        zeroes_probabilities = [1 - p for p in ones_probabilities]
        probabilities = np.array([zeroes_probabilities, ones_probabilities])
        return probabilities.transpose()

In [80]:
my_ensemble = MyEnsemble(**clf.best_params_, bootstrap=True)

In [81]:
my_ensemble.fit(train_X, train_y)

[LightGBM] [Warning] Unknown parameter: min_impurity_decrease
[LightGBM] [Warning] Unknown parameter: min_weight_fraction_leaf
[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: max_samples
[LightGBM] [Warning] Unknown parameter: bootstrap
[LightGBM] [Warning] min_data_in_leaf is set with min_samples_leaf=39, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=39
[LightGBM] [Warning] Unknown parameter: criterion
[LightGBM] [Warning] num_leaves is set=31, max_leaf_nodes=123 will be ignored. Current value: num_leaves=31


In [82]:
y_pred = my_ensemble.predict(valid_X)

In [83]:
accuracy_score(valid_y, y_pred)

0.5321624477245551

In [84]:
y_pred = my_ensemble.predict_proba(valid_X)

In [85]:
roc_auc_score(valid_y, y_pred[:, 1])

0.551326180384595

Точность вышла меньше чем например у lightgbm из-за того что точность двух других класификаторов очень мала

In [86]:
# y_pred = lightgbm_rfc.predict_proba(impured_test)
# y_pred = rfc_sklearn.predict_proba(impured_test)
y_pred = my_clf.predict_proba(impured_test)
# y_pred = my_ensemble.predict_proba(impured_test)

In [87]:
y_pred = pd.DataFrame(data={
    'id': test_data.id, 
    'claim': y_pred[:, 1]}, index=impured_test.index)

In [88]:
y_pred

,id,claim
0,957919,0.471283
1,957920,0.476165
2,957921,0.344484
3,957922,0.574075
4,957923,0.446240
...,...,...
493469,1451388,0.000000
493470,1451389,0.000000
493471,1451390,0.000000
493472,1451391,0.000000


In [89]:
pd.DataFrame(y_pred).to_csv('my_clf_solution.csv',
        columns=['id', 'claim'], index=False)

Kaggle scores:

lightgbm - 0.67093, 
sklearn - 0.58600, 
my_clf - 0.50000,
my_ensemble - 0.56684